In [ ]:
print("test")

In [11]:
import cv2
import numpy as np
from line_method import click_event, coordinates_on_line
##linmethod: pointa = 1766, 1782; pointb = 1928, 1916
# Read RGB image
img = cv2.imread('I:\\2023_04_06_PLMA_HexaDecane_Basler2x_Xp1_24_s11_split____GOODHALO-DidntReachSplit\\D_analysis_v2\PROC_20230612121104\\rawslicesimage\\rawslicesimage_Basler_a2A5328-15ucBAS__40087133__20230406_131652896_0023_analyzed_.png')


#imgblack = cv2.imread('C:\\Users\\Sander PC\\PycharmProjects\\InterferometryPython\\red square.png')
imgblack = cv2.imread('C:\\Users\\ReuvekampSW\\PycharmProjects\\InterferometryPython\\red square.png')
resizedimg = cv2.resize(img, [2400, 1500], interpolation = cv2.INTER_AREA)
squareSize = 10
resizedImagBlack = cv2.resize(imgblack, [squareSize, squareSize], interpolation = cv2.INTER_AREA)

pixellocationLarge = 2250
pixellocationLarge2 = 2950

#x_offset = 468
#y_offset = 114

a = 0.827846
b = -183.776
limits = [466, 1937, 112, 1385]     #xmin xmin ymin ymax
coordinates, l = coordinates_on_line(a, b, limits)      #INCORRECT VGM. ZELF BEREKEND=1852
print(f"Length of line is {l}. (Image in plot)")

x_coords = [1766, 1928]
y_coords = [1782, 1916]
aL = (y_coords[1]-y_coords[0])/(x_coords[1]-x_coords[0])
bL = y_coords[0] - a * x_coords[0]
print(f"(Large plot). a ={aL}, b = {bL}")
limitsL = [0, 5328, 0, 4608]
coordinatesLarge, lLarge = coordinates_on_line(aL, bL, limitsL)
print(f"Length of line is {lLarge}. (Large plot)")

ratioLines = lLarge / 1852
pixelLocNew = pixellocationLarge / ratioLines
print(f"ratioLines = {ratioLines}, pixelLocLarge = {pixellocationLarge}, pixelLocNew = {pixelLocNew}")
#pixely = round(a*(pixelLocNew+466) + b)

#x_offset = round(pixelLocNew)  + 466
#y_offset = pixely

c = pixelLocNew     #c = length of line (schuine zijde)
#x_offset = round((-a*b + np.sqrt(c**2 + a**2 * c**2 - b**2)) / (1+a**2))
x_offset = round(c / (np.sqrt(1+a**2)) + 466 - squareSize/2)            #-squareSize/2 to centre it
pixely = round(a*(x_offset) + b)
y_offset = pixely
print(f"pixelLocNew={pixelLocNew} x={x_offset}, y = {y_offset}")

resizedimg[y_offset:y_offset+resizedImagBlack.shape[0], x_offset:x_offset+resizedImagBlack.shape[1]] = resizedImagBlack
cv2.imshow('image', resizedimg)
right_clicks = []
def click_event(event, x, y, flags, params):
    if event == cv2.EVENT_LBUTTONDOWN:
        right_clicks.append([x, y])
    if len(right_clicks) == 2:
        cv2.destroyAllWindows()

cv2.setMouseCallback('image', click_event)
cv2.waitKey(0)

P1 = np.array(right_clicks[0])
P2 = np.array(right_clicks[1])
print(f"Selected coordinates: {P1=}, {P2=}.")
print(f"Selected coordinates: P1 = [{P1[0]:.0f}, {P1[1]:.0f}], P2 = [{P2[0]:.0f}, {P2[1]:.0f}]")

cv2.destroyAllWindows()


Length of line is 2172.0. (Image in plot)
(Large plot). a =0.8271604938271605, b = 320.023964
Length of line is 6727.57713524293. (Large plot)
ratioLines = 3.6326010449475863, pixelLocLarge = 2250, pixelLocNew = 619.39089158426
pixelLocNew=619.39089158426 x=938, y = 593
Selected coordinates: P1=array([471, 203]), P2=array([1067,  697]).
Selected coordinates: P1 = [471, 203], P2 = [1067, 697]


In [5]:
import glob
import os.path
import cv2
import numpy as np
import csv
from line_method import click_event, coordinates_on_line
""""
This part is to show dots of pixellocations for all swellingImages, without clickingevents (below; to obtain required sizes in pixels etc.).
"""
##linmethod: pointa = 1766, 1782; pointb = 1928, 1916
# Read RGB image
source = 'D:\\2023_04_06_PLMA_HexaDecane_Basler2x_Xp1_24_s11_split____GOODHALO-DidntReachSplit\\D_analysis_v2\PROC_20230612121104'
imgList = [f for f in glob.glob(os.path.join(source, f"rawslicesimage\\*.png"))]
pixellocationLarge = [0, 2250, 2550, 2950, 3800, 6707]#2170
CLICKEVENT = False
n = 0
#nAllImages = np.arange(0, len(imgList),1)
nAllImages = [0,1]
for imgPath in imgList:
    if n in nAllImages:
        #img = cv2.imread(os.path.join(source, 'rawslicesimage\\rawslicesimage_Basler_a2A5328-15ucBAS__40087133__20230217_110816090_0009_analyzed_.png'))
        img = cv2.imread(imgPath)
        csvList = [f for f in glob.glob(os.path.join(source, f"csv\\*unwrapped.csv"))]

        #imgblack = cv2.imread('C:\\Users\\ReuvekampSW\\Documents\\InterferometryPython\\red square.png')
        imgblack = cv2.imread('C:\\Users\\ReuvekampSW\\PycharmProjects\\InterferometryPython\\red square.png')
        resizedimg = cv2.resize(img, [2400, 1500], interpolation = cv2.INTER_AREA)
        squareSize = 5
        resizedImagBlack = cv2.resize(imgblack, [squareSize, squareSize], interpolation = cv2.INTER_AREA)

        def calcLineEquation(x_coords, y_coords):       #For eq   y = ax +b
            aL = (y_coords[1]-y_coords[0])/(x_coords[1]-x_coords[0])    #a = dy /dx
            bL = y_coords[0] - aL * x_coords[0]                          #b = y(x) - a*x
            return aL, bL
        def calcLineLength(edge1, edge2):   #edge1(x,y)    edge2(x,y)
            return ((edge2[0] - edge1[0])**2 + (edge2[1] - edge1[1])**2 )**0.5
        def readInDataLengthLargeImage(csvList):
            with open(csvList[0]) as f:
                summation = sum(1 for line in f)
            return summation-1

        #P1 and P2 along the draw line of image in plot
        P1 = [466, 206]
        P2 = [1091, 725]
        a, b = calcLineEquation([P1[0], P2[0]], [P1[1], P2[1]])
        limits = [466, 1937, 112, 1385]     #xmin xmin ymin ymax of image in plot. Should always be same
        l = calcLineLength(P1, P2)
        print(f"Image in plot: length calculated is {l} from edges")

        x_coords = [1766, 1782]
        y_coords = [1928, 1916]
        aL, bL = calcLineEquation(x_coords, y_coords)
        print(f"(Large plot). a ={aL}, b = {bL}")
        lLarge = readInDataLengthLargeImage(csvList)    #read in OG image data length of line from counting rows in csv file

        ratioLines = lLarge / l
        for pixelLocation in pixellocationLarge:
            pixelLocNew = pixelLocation / ratioLines
            print(f"ratioLines = {ratioLines}, pixelLocLarge = {pixelLocation}, pixelLocNew = {pixelLocNew}")

            c = pixelLocNew     #c = length of line (schuine zijde)
            x_offset = round(c / (np.sqrt(1+a**2)) + 466 - squareSize/2)            #-squareSize/2 to centre it
            pixely = round(a*(x_offset) + b)
            y_offset = pixely
            print(f"pixelLocNew={pixelLocNew} x={x_offset}, y = {y_offset}")
            resizedimg[y_offset:y_offset+resizedImagBlack.shape[0], x_offset:x_offset+resizedImagBlack.shape[1]] = resizedImagBlack

            if CLICKEVENT:
                cv2.imshow('image', resizedimg)
                right_clicks = []
                def click_event(event, x, y, flags, params):
                    if event == cv2.EVENT_LBUTTONDOWN:
                        right_clicks.append([x, y])
                    if len(right_clicks) == 2:
                        cv2.destroyAllWindows()

                cv2.setMouseCallback('image', click_event)
                cv2.waitKey(0)
                P1 = np.array(right_clicks[0])
                P2 = np.array(right_clicks[1])
                print(f"Selected coordinates: {P1=}, {P2=}.")
                print(f"Selected coordinates: P1 = [{P1[0]:.0f}, {P1[1]:.0f}], P2 = [{P2[0]:.0f}, {P2[1]:.0f}]")
                cv2.destroyAllWindows()
        if not os.path.exists(os.path.join(source, f"rawslicesimage\\pixelLocation")):
            os.mkdir(os.path.join(source, f"rawslicesimage\\pixelLocation"))
        cv2.imwrite(os.path.join(source, f"rawslicesimage\\pixelLocation\\rawlsiceimageWithPixelLocation{n}.png") , resizedimg)
    n = n+1


Image in plot: length calculated is 812.395224013534 from edges
(Large plot). a =-0.75, b = 3252.5
ratioLines = 8.254602934357333, pixelLocLarge = 0, pixelLocNew = 0.0
pixelLocNew=0.0 x=464, y = 204
ratioLines = 8.254602934357333, pixelLocLarge = 2250, pixelLocNew = 272.5751944572698
pixelLocNew=272.5751944572698 x=673, y = 378
ratioLines = 8.254602934357333, pixelLocLarge = 2550, pixelLocNew = 308.91855371823914
pixelLocNew=308.91855371823914 x=701, y = 401
ratioLines = 8.254602934357333, pixelLocLarge = 2950, pixelLocNew = 357.3763660661982
pixelLocNew=357.3763660661982 x=738, y = 432
ratioLines = 8.254602934357333, pixelLocLarge = 3800, pixelLocNew = 460.3492173056112
pixelLocNew=460.3492173056112 x=818, y = 498
ratioLines = 8.254602934357333, pixelLocLarge = 6707, pixelLocNew = 812.5163685444038
pixelLocNew=812.5163685444038 x=1089, y = 723
Image in plot: length calculated is 812.395224013534 from edges
(Large plot). a =-0.75, b = 3252.5
ratioLines = 8.254602934357333, pixelLocLarg